In [ ]:
from tensorflow.keras.applications import MobileNetV2
import cv2
import re
from os import walk
import pandas as pd
import imutils
import numpy as np
import os
import random
import shutil

import keras
from tensorflow.keras.optimizers import Adam, RMSprop, Nadam
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
#from tensorflow.keras.layers.normalization import BatchNormalization
from keras.models import Model
from scipy.spatial.distance import cosine
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Data Cleaning

In [ ]:
!kaggle datasets download -d wobotintelligence/face-mask-detection-dataset

 99% 2.48G/2.50G [00:37<00:00, 58.7MB/s]
100% 2.50G/2.50G [00:37<00:00, 71.8MB/s]


In [ ]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

 96% 318M/330M [00:03<00:00, 101MB/s] 
100% 330M/330M [00:03<00:00, 97.9MB/s]


In [ ]:
!unzip -q face-mask-12k-images-dataset

In [ ]:
uses = ['Train', 'Test']
for use in uses:
  groups = ['WithMask', 'WithoutMask']
  for group in groups:
    a = []
    for (dirpath, dirnames, filenames) in walk('/content/Face Mask Dataset/{}/{}'.format(use, group)):
        a.extend(filenames)
        break
    for name in a:
      os.rename('/content/Face Mask Dataset/{}/{}/{}'.format(use, group, name), '/content/Face Mask Dataset/{}/{}/data2_{}'.format(use, group, name))


In [ ]:
!unzip -q /content/face-mask-detection-dataset.zip -d data

In [ ]:
mask_pos = ['face_with_mask']
using = ['face_with_mask', 'face_no_mask', 'face_with_mask_incorrect']
df = pd.read_csv('/content/data/train.csv')
df = df.loc[(df['classname'].isin(using))]

df['mask'] = df['classname'].map(lambda x: x in mask_pos)
df['temp'] = df['x2']
df['x2']= df['y1']
df['y1'] = df['temp']
del df['temp']

df['pred_x1'], df['pred_y1'], df['pred_x2'], df['pred_y2'] = 0, 0, 0, 0
df.head(10)

,name,x1,x2,y1,y2,classname,mask,pred_x1,pred_y1,pred_x2,pred_y2
0,2756.png,69,294,126,392,face_with_mask,True,0,0,0,0
1,2756.png,505,723,10,283,face_with_mask,True,0,0,0,0
4,6098.jpg,360,728,85,653,face_no_mask,False,0,0,0,0
5,6427.png,218,577,98,580,face_with_mask_incorrect,False,0,0,0,0
7,4591.png,239,522,9,342,face_with_mask,True,0,0,0,0
11,5525.jpg,262,484,18,319,face_no_mask,False,0,0,0,0
12,5525.jpg,49,191,58,247,face_no_mask,False,0,0,0,0
13,3911.png,49,389,26,476,face_no_mask,False,0,0,0,0
16,4287.png,80,629,0,599,face_with_mask,True,0,0,0,0
17,4893.png,179,664,8,599,face_with_mask,True,0,0,0,0


In [ ]:
!wget -N https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
!wget -N https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel

--2021-03-13 21:04:48--  https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28104 (27K) [text/plain]
Saving to: ‘deploy.prototxt’

deploy.prototxt     100%[===================>]  27.45K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2021-03-13 21:04:49 (145 MB/s) - ‘deploy.prototxt’ saved [28104/28104]

--2021-03-13 21:04:49--  https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubuse

In [ ]:
prototxt = 'deploy.prototxt'
model_path = 'res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt, model_path)
def get_face(image):
  (h, w) = image.shape[:2]

  blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
  net.setInput(blob)
  detections = net.forward()

  faces = np.zeros((detections.shape[2], 4))
  for i in range(0, detections.shape[2]):
    # extract the confidence (i.e., probability) associated with the prediction
    confidence = detections[0, 0, i, 2]

    if confidence > 0.3:
      box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
      faces[i] = box
  return faces

def compare_faces(dbox, pbox, size):
  darea = abs((1 + dbox[2] - dbox[0])*(1 + dbox[3]-dbox[1]))
  parea = abs((1 + pbox[2] - pbox[0])*(1 + pbox[3]-pbox[1]))
#  print(sum(abs(dbox-pbox)), darea*.008)
  if sum(abs(dbox-pbox)) <= .008*darea and (parea * .7 >= darea or darea * .7 >= parea):
    return True
  pbox = np.array([pbox[2], pbox[3], pbox[0], pbox[1]])
  if sum(abs(dbox-pbox)) <= .008*darea:
    return True
  return False

In [ ]:

all_names = []
for (dirpath, dirnames, filenames) in walk('/content/data/Medical mask/Medical mask/Medical Mask/images'):
    all_names.extend(filenames)
    break
print(len(all_names))
print(all_names[:10])

6024
['6238.jpg', '5266.jpg', '4818.png', '0793.jpg', '4282.png', '2781.png', '3912.png', '5205.jpg', '5184.jpg', '0291.jpg']


In [ ]:
j = 0
#all_names = ['4591.png', '4287.png', '5525.jpg', '5010.jpg']
for image_name in all_names:
#  print('\n')
  j+=1
  if (j%50 == 0):
    print(j)
  image = cv2.imread('/content/data/Medical mask/Medical mask/Medical Mask/images/{}'.format(image_name))
  pred_faces = get_face(image)
  pred_faces = [x for x in pred_faces if np.sum(x) != 0]
  idx = df.index[(df['name'] == image_name)].to_numpy()
  for index in idx:
    dataset_box = [df.loc[index]['x1'], df.loc[index]['y1'], df.loc[index]['x2'], df.loc[index]['y2']]
    for pred_box in pred_faces:
      if compare_faces(dataset_box, pred_box, image.shape[0]*image.shape[1]):
#        print(pred_box)
#        cv2.rectangle(image, (dataset_box[0], dataset_box[1]), (dataset_box[2], dataset_box[3]), (255, 0, 0), 2)
#        cv2.rectangle(image, (int(pred_box[0]), int(pred_box[1])), (int(pred_box[2]), int(pred_box[3])), (0, 255, 0), 2)
#        cv2.imwrite('img4.png', image)
        df['pred_x1'][index], df['pred_y1'][index], df['pred_x2'][index], df['pred_y2'][index] = pred_box


In [ ]:
df.head()

,name,x1,x2,y1,y2,classname,mask,pred_x1,pred_y1,pred_x2,pred_y2
0,2756.png,69,294,126,392,face_with_mask,True,90,161,273,384
1,2756.png,505,723,10,283,face_with_mask,True,0,0,0,0
4,6098.jpg,360,728,85,653,face_no_mask,False,0,0,0,0
5,6427.png,218,577,98,580,face_with_mask_incorrect,False,221,141,535,520
7,4591.png,239,522,9,342,face_with_mask,True,0,0,0,0


In [ ]:
#OPEN CV
try:
  os.makedirs('/content/data2/train/mask')
  os.makedirs('/content/data2/train/no_mask')
  os.makedirs('/content/data2/test/mask')
  os.makedirs('/content/data2/test/no_mask')
except: pass

for index, row in df.iterrows():
   image = cv2.imread('/content/data/Medical mask/Medical mask/Medical Mask/images/{}'.format(row['name']))
   if (row['pred_x1'] + row['pred_y1'] + row['pred_x2'] + row['pred_y2'] != 0):
     image = image[row['pred_y1']:row['pred_y2'], row['pred_x1']:row['pred_x2']]
     if row['mask']:
       cv2.imwrite('/content/data2/train/mask/{}'.format(row['name']), image)
     else:
       cv2.imwrite('/content/data2/train/no_mask/{}'.format(row['name']), image)       

In [ ]:
#ANNOTATED

try:
  os.makedirs('/content/data2/train/mask')
  os.makedirs('/content/data2/train/no_mask')
  os.makedirs('/content/data2/test/mask')
  os.makedirs('/content/data2/test/no_mask')
except: pass

for index, row in df.iterrows():
  image = cv2.imread('/content/data/Medical mask/Medical mask/Medical Mask/images/{}'.format(row['name']))
  image = image[row['y1']:row['y2'], row['x1']:row['x2']]
  try:
    if row['mask']:
      cv2.imwrite('/content/data2/train/mask/{}'.format(row['name']), image)
    else:
      cv2.imwrite('/content/data2/train/no_mask/{}'.format(row['name']), image)
  except:
    print(row)

name                         1861.jpg
x1                                582
x2                                582
y1                                126
y2                                126
classname    face_with_mask_incorrect
mask                            False
pred_x1                             0
pred_y1                             0
pred_x2                             0
pred_y2                             0
Name: 8131, dtype: object


In [ ]:
a = []
for (dirpath, dirnames, filenames) in walk('/content/data2/train/mask'):
    a.extend(filenames)
    break
random.shuffle(a)

for x in a[:len(a)//5]:
  shutil.move('/content/data2/train/mask/{}'.format(x), '/content/data2/test/mask')

a = []
for (dirpath, dirnames, filenames) in walk('/content/data2/train/no_mask'):
    a.extend(filenames)
    break
random.shuffle(a)

for x in a[:len(a)//5]:
  shutil.move('/content/data2/train/no_mask/{}'.format(x), '/content/data2/test/no_mask')

# Training

In [ ]:
a = []
for (dirpath, dirnames, filenames) in walk('/content/data2/train'):
    a.extend(dirnames)
    break
print(a)

a = []
for (dirpath, dirnames, filenames) in walk('/content/data2/test'):
    a.extend(dirnames)
    break
print(a)

['no_mask', 'mask']
['no_mask', 'mask']


In [ ]:
os.rmdir('/content/data2/train/.ipynb_checkpoints')
os.rmdir('/content/data2/test/.ipynb_checkpoints')

In [ ]:
train_data_dir = '/content/data2/train'
validation_data_dir = '/content/data2/test'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
#      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator()#rescale=1./255)

# Set the Batch Size according to your system.
train_batchsize = 128
val_batchsize = 32

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 2964 images belonging to 2 classes.
Found 740 images belonging to 2 classes.


In [ ]:
model = Sequential()
vg = VGG16(weights = 'imagenet')

for layer in vg.layers[:-3]:
  layer.trainable = False
  model.add(layer)
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256))
model.add(Dense(256))
model.add(Dense(256))
model.add(Dense(2, activation = 'softmax'))
model.summary()
model.compile(optimizer=Adam(), loss = 'categorical_crossentropy', metrics = ['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [ ]:
my_callbacks = [
    keras.callbacks.ModelCheckpoint(
    '/content/mask_checkpoint_{val_accuracy:.3f}.keras',
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="max",
    save_freq="epoch",
    options=None,
),
]

In [ ]:
model.fit(train_generator,  epochs=100, validation_data=validation_generator, callbacks=my_callbacks)

Epoch 1/100
24/24 [==============================] - 73s 2s/step - loss: 51.6663 - accuracy: 0.5917 - val_loss: 1.0640 - val_accuracy: 0.8527

Epoch 00001: val_accuracy improved from -inf to 0.85270, saving model to /content/mask_checkpoint_0.853.keras
Epoch 2/100
24/24 [==============================] - 44s 2s/step - loss: 0.5371 - accuracy: 0.8618 - val_loss: 0.3043 - val_accuracy: 0.8878

Epoch 00002: val_accuracy improved from 0.85270 to 0.88784, saving model to /content/mask_checkpoint_0.888.keras
Epoch 3/100
24/24 [==============================] - 44s 2s/step - loss: 0.2868 - accuracy: 0.8846 - val_loss: 0.2885 - val_accuracy: 0.8784

Epoch 00003: val_accuracy did not improve from 0.88784
Epoch 4/100
24/24 [==============================] - 45s 2s/step - loss: 0.2597 - accuracy: 0.8926 - val_loss: 0.2783 - val_accuracy: 0.9027

Epoch 00004: val_accuracy improved from 0.88784 to 0.90270, saving model to /content/mask_checkpoint_0.903.keras
Epoch 5/100
24/24 [=====================

In [ ]:
model.save('mask.keras')

In [ ]:
shutil.copy('/content/mask_checkpoint_0.926.keras', '/content/drive/My Drive/my_models/mask_given.keras')

'/content/drive/My Drive/my_models/mask_given.keras'

# Testing

In [ ]:
model = keras.models.load_model('mask.keras')

In [ ]:
!wget -N https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
!wget -N https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel

--2022-01-16 17:00:30--  https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28104 (27K) [text/plain]
Saving to: ‘deploy.prototxt’

deploy.prototxt     100%[===================>]  27.45K  --.-KB/s    in 0.002s  

Last-modified header missing -- time-stamps turned off.
2022-01-16 17:00:30 (10.9 MB/s) - ‘deploy.prototxt’ saved [28104/28104]

--2022-01-16 17:00:30--  https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubus

In [ ]:
prototxt_path = 'deploy.prototxt'
model_path = 'res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)
def get_face(image, threshold=.2):
  (h, w) = image.shape[:2]

  blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
  net.setInput(blob)
  detections = net.forward()

  faces = np.zeros((detections.shape[2], 4))
  for i in range(0, detections.shape[2]):
    # extract the confidence (i.e., probability) associated with the prediction
    confidence = detections[0, 0, i, 2]

    if confidence > threshold:
      box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
      faces[i] = box
  return faces


In [ ]:
img_name = 'masks5.jpg'

In [ ]:
# First one takes some time
image = cv2.imread(img_name)
write_img = np.array(image)
faces = get_face(image)
faces = [x for x in faces if np.sum(x) != 0]
for face in faces:
  face = face.astype(int)
  new_img = image[face[1]:face[3], face[0]:face[2]]
  new_img = cv2.resize(new_img, (224, 224))
#  new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
  new_img = np.reshape(new_img, (1, 224, 224, 3))
  pred = model.predict(new_img)[0]
  if np.argmax(pred) == 0:
    text = 'mask'
    cv2.putText(write_img, text, (face[0], face[1]), cv2.FONT_HERSHEY_SIMPLEX, .6, (0, 255, 0), 2)
    cv2.rectangle(write_img, (face[0], face[1]), (face[2], face[3]), (0, 255, 0), 2)

  else:
    text = 'no_mask'
    cv2.putText(write_img, text, (face[0], face[1]), cv2.FONT_HERSHEY_SIMPLEX, .6, (0, 0, 255), 2)
    cv2.rectangle(write_img, (face[0], face[1]), (face[2], face[3]), (0, 0, 255), 2)
cv2.imwrite('outputs/output.png', write_img)

True